In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import concurrent.futures
import jinja2
from anthropic import AnthropicBedrock
from compiler.core import Compiler
from tracing_client import TracingClient
from policies import common, handlers, handler_tests, typespec, drizzle, typescript, app_testcases

In [2]:
jinja_env = jinja2.Environment()
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = TracingClient(AnthropicBedrock(aws_profile="dev", aws_region="us-west-2"))

In [3]:
application_description = "Bot that greets user by name and returns personalized greeting."

content = jinja_env.from_string(typespec.PROMPT).render(application_description=application_description)
message = {"role": "user", "content": content}
with typespec.TypespecTaskNode.platform(client, compiler, jinja_env):
    tsp_data = typespec.TypespecTaskNode.run([message])
    tsp_root = typespec.TypespecTaskNode(tsp_data)
    tsp_solution = tsp_root
    tsp_solution = common.bfs(tsp_root)

match tsp_solution.is_successful:
    case True:
        print("Typespec solution is successful.")
    case False:
        raise Exception("Typespec solution is not successful.")

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Typespec solution is successful.


In [4]:
print(tsp_solution.data.output.typespec_definitions)

model GreetingOptions {
    userName: string;
    language: string;
    timeOfDay: plainTime;
    formal: boolean;
}

model GreetingResponse {
    message: string;
    timestamp: utcDateTime;
}

interface GreetingBot {
    @llm_func("Generate personalized greeting for the user")
    greet(options: GreetingOptions): GreetingResponse;
}


In [5]:
content = jinja_env.from_string(typescript.PROMPT).render(typespec_definitions=tsp_solution.data.output.typespec_definitions)
message = {"role": "user", "content": content}
with typescript.TypescriptTaskNode.platform(client, compiler, jinja_env):
    ts_data = typescript.TypescriptTaskNode.run([message])
    ts_root = typescript.TypescriptTaskNode(ts_data)
    ts_solution = common.bfs(ts_root)

match ts_solution.is_successful:
    case True:
        print("Typescript solution is successful.")
    case False:
        raise Exception("Typescript solution is not successful.")

Typescript solution is successful.


In [6]:
ts_solution.data.output.__dict__

{'reasoning': "The application is a greeting bot that generates personalized greetings based on various parameters:\n- User's name\n- Preferred language\n- Time of day\n- Formality level\nThe response includes the greeting message and a timestamp of when it was generated.\nThe time-related fields use specific time formats: plainTime for time of day and utcDateTime for timestamp.",
 'typescript_schema': "import { z } from 'zod';\n\nexport const greetingOptionsSchema = z.object({\n    userName: z.string(),\n    language: z.string(),\n    timeOfDay: z.string().transform((str) => z.coerce.date().parse(str)),\n    formal: z.boolean()\n});\n\nexport type GreetingOptions = z.infer<typeof greetingOptionsSchema>;\n\nexport const greetingResponseSchema = z.object({\n    message: z.string(),\n    timestamp: z.string().transform((str) => z.coerce.date().parse(str))\n});\n\nexport type GreetingResponse = z.infer<typeof greetingResponseSchema>;\n\nexport declare function greet(options: GreetingOptio

In [7]:
print(ts_solution.data.output.typescript_schema)

import { z } from 'zod';

export const greetingOptionsSchema = z.object({
    userName: z.string(),
    language: z.string(),
    timeOfDay: z.string().transform((str) => z.coerce.date().parse(str)),
    formal: z.boolean()
});

export type GreetingOptions = z.infer<typeof greetingOptionsSchema>;

export const greetingResponseSchema = z.object({
    message: z.string(),
    timestamp: z.string().transform((str) => z.coerce.date().parse(str))
});

export type GreetingResponse = z.infer<typeof greetingResponseSchema>;

export declare function greet(options: GreetingOptions): Promise<GreetingResponse>;


In [8]:
content = jinja_env.from_string(drizzle.PROMPT).render(typespec_definitions=tsp_solution.data.output.typespec_definitions)
message = {"role": "user", "content": content}
with drizzle.DrizzleTaskNode.platform(client, compiler, jinja_env):
    dzl_data = drizzle.DrizzleTaskNode.run([message])
    dzl_root = drizzle.DrizzleTaskNode(dzl_data)
    dzl_solution = common.bfs(dzl_root)

match dzl_solution.is_successful:
    case True:
        print("Drizzle solution is successful.")
    case False:
        raise Exception("Drizzle solution is not successful.")

Drizzle solution is successful.


In [9]:
print(dzl_solution.data.output.drizzle_schema)

import { integer, pgTable, text, timestamp } from "drizzle-orm/pg-core";

// Store greeting requests
export const greetingRequestsTable = pgTable("greeting_requests", {
  id: integer().primaryKey().generatedAlwaysAsIdentity(),
  name: text().notNull(),
  language: text().notNull(),
  created_at: timestamp().defaultNow().notNull()
});

// Store greeting responses
export const greetingResponsesTable = pgTable("greeting_responses", {
  id: integer().primaryKey().generatedAlwaysAsIdentity(),
  request_id: integer().references(() => greetingRequestsTable.id),
  response_text: text().notNull(),
  created_at: timestamp().defaultNow().notNull()
});


In [10]:
handler_target = ts_solution.data.output.functions[0]

In [11]:
def search_test(
    function_name: str,
    typescript_schema: str,
    drizzle_schema: str,
) -> handler_tests.HandlerTestTaskNode:
    prompt_params = {
        "function_name": function_name,
        "typescript_schema": typescript_schema,
        "drizzle_schema": drizzle_schema,
    }
    content = jinja_env.from_string(handler_tests.PROMPT).render(**prompt_params)
    message = {"role": "user", "content": content}
    test_data = handler_tests.HandlerTestTaskNode.run([message], **prompt_params)
    test_root = handler_tests.HandlerTestTaskNode(test_data)
    test_solution = common.bfs(test_root, **prompt_params)
    return test_solution

In [12]:
with handler_tests.HandlerTestTaskNode.platform(client, compiler, jinja_env):
    ht_solution = search_test(
        handler_target.name,
        ts_solution.data.output.typescript_schema,
        dzl_solution.data.output.drizzle_schema,
    )

In [13]:
ht_solution.data.output.__dict__

{'imports': 'import { expect, it } from "bun:test";\nimport { db } from "../../db";\nimport { greetingRequestsTable, greetingResponsesTable } from "../../db/schema/application";\nimport { type GreetingRequest } from "../../common/schema";\nimport { eq } from "drizzle-orm";',
 'tests': ['\nit("should return a greeting message", async () => {\n  const input: GreetingRequest = { name: "Alice", language: "en" };\n  const result = await greeting(input);\n  expect(typeof result).toBe("string");\n  expect(result.length).toBeGreaterThan(0);\n});\n',
  '\nit("should store the greeting request in database", async () => {\n  const input: GreetingRequest = { name: "Bob", language: "es" };\n  await greeting(input);\n  \n  const requests = await db.select().from(greetingRequestsTable).where(\n    eq(greetingRequestsTable.name, "Bob")\n  ).execute();\n  \n  expect(requests).toHaveLength(1);\n  expect(requests[0].name).toBe("Bob");\n  expect(requests[0].language).toBe("es");\n});\n',
  '\nit("should s

In [14]:
for test in ht_solution.data.output.tests:
    print(test)


it("should return a greeting message", async () => {
  const input: GreetingRequest = { name: "Alice", language: "en" };
  const result = await greeting(input);
  expect(typeof result).toBe("string");
  expect(result.length).toBeGreaterThan(0);
});


it("should store the greeting request in database", async () => {
  const input: GreetingRequest = { name: "Bob", language: "es" };
  await greeting(input);
  
  const requests = await db.select().from(greetingRequestsTable).where(
    eq(greetingRequestsTable.name, "Bob")
  ).execute();
  
  expect(requests).toHaveLength(1);
  expect(requests[0].name).toBe("Bob");
  expect(requests[0].language).toBe("es");
});


it("should store both request and response", async () => {
  const input: GreetingRequest = { name: "Carol", language: "fr" };
  const response = await greeting(input);
  
  const requests = await db.select().from(greetingRequestsTable).where(
    eq(greetingRequestsTable.name, "Carol")
  ).execute();
  
  expect(requests).toHave

In [15]:
test_suite = jinja_env.from_string(handler_tests.HANDLER_TEST_TPL).render(
    handler_name=handler_target.name,
    handler_function_import=f'import {{ handle as {handler_target.name} }} from "../../handlers/{handler_target.name}.ts";',
    imports=ht_solution.data.output.imports,
    tests=ht_solution.data.output.tests,
)

In [19]:
def search_handler(
    function_name: str,
    argument_type: str,
    argument_schema: str,
    typespec_definitions: str,
    typescript_schema: str,
    drizzle_schema: str,
    test_suite: str,
) -> handlers.HandlerTaskNode:
    prompt_params = {
        "function_name": function_name,
        "argument_type": argument_type,
        "argument_schema": argument_schema,
        "typespec_schema": typespec_definitions,
        "typescript_schema": typescript_schema,
        "drizzle_schema": drizzle_schema,
        "test_suite": test_suite,
    }
    content = jinja_env.from_string(handlers.PROMPT).render(**prompt_params)
    message = {"role": "user", "content": content}
    output = handlers.HandlerTaskNode.run([message], **prompt_params)
    root_node = handlers.HandlerTaskNode(output)
    #return root_node
    solution = common.bfs(root_node, **prompt_params)
    return solution

In [22]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    handler_solution = search_handler(
        function_name=handler_target.name,
        argument_type=handler_target.argument_type,
        argument_schema=handler_target.argument_schema,
        typespec_definitions=tsp_solution.data.output.typespec_definitions,
        typescript_schema=ts_solution.data.output.typescript_schema,
        drizzle_schema=dzl_solution.data.output.drizzle_schema,
        test_suite=test_suite,
    )

In [23]:
handler_solution.data.output.__dict__

{'name': 'greetingHandler',
 'handler': 'import { db } from "../db";\nimport { type GreetingRequest, greeting } from "../common/schema";\nimport { greetingRequestsTable, greetingResponsesTable } from "../db/schema/application";\n\nconst getGreetingByLanguage = (name: string, language: string): string => {\n    switch (language.toLowerCase()) {\n        case "es":\n            return `¡Hola, ${name}!`;\n        case "fr":\n            return `Bonjour, ${name}!`;\n        default:\n            return `Hello, ${name}!`;\n    }\n};\n\nexport const handle: typeof greeting = async (options: GreetingRequest): Promise<string> => {\n    // Generate greeting message\n    const greetingMessage = getGreetingByLanguage(options.name, options.language);\n    \n    // Store request and get the request id\n    const [request] = await db.insert(greetingRequestsTable)\n        .values({\n            name: options.name,\n            language: options.language,\n        })\n        .returning();\n\n    // 

In [24]:
print(handler_solution.data.output.test_feedback["stderr"])


src/tests/handlers/greeting.test.ts:
(pass) greeting > should return a greeting message [3.00ms]
(pass) greeting > should store the greeting request in database [2.00ms]
(pass) greeting > should store both request and response [2.00ms]

 3 pass
 0 fail
 8 expect() calls
Ran 3 tests across 1 files. [243.00ms]



In [25]:
import re

In [26]:
fail_pass_pat = re.compile(r"(\d+) pass\s+(\d+) fail", re.MULTILINE)

In [27]:
test_case = """
src/tests/handlers/greeting.test.ts:
(pass) greeting > should return a greeting message [3.00ms]
(pass) greeting > should store the greeting request in database [2.00ms]
(pass) greeting > should store both request and response [2.00ms]

 3 pass
 0 fail
 8 expect() calls
Ran 3 tests across 1 files. [243.00ms]
"""

In [28]:
matches = fail_pass_pat.search(test_case)

In [30]:
matches.groups()

('3', '0')

In [24]:
handler_solution.is_successful

True

In [15]:
handler_solution.data.output.__dict__

{'name': 'greetHandler',
 'handler': 'import { db } from "../db";\nimport { greet, type GreetingRequest, type GreetingResponse } from "../common/schema";\nimport { greetingRequestsTable, greetingResponsesTable } from "../db/schema/application";\n\nexport const handle: typeof greet = async (options: GreetingRequest): Promise<GreetingResponse> => {\n    // Insert the greeting request into the database\n    const [requestResult] = await db\n        .insert(greetingRequestsTable)\n        .values({\n            name: options.name,\n        })\n        .returning();\n\n    // Create greeting message\n    const message = `Hello, ${options.name}!`;\n\n    // Store the response\n    await db\n        .insert(greetingResponsesTable)\n        .values({\n            request_id: requestResult.id,\n            message: message,\n        });\n\n    // Return the greeting response\n    return {\n        message: message\n    };\n};',
 'feedback': {'exit_code': 0, 'stdout': None, 'stderr': None},
 'te

In [18]:
handler_solution.data.output

ValueError('too many values to unpack (expected 2)')

In [ ]:
print(test_suite)

import { afterEach, beforeEach, describe } from "bun:test";
import { resetDB, createDB } from "../../helpers";
import { handle as greet } from "../../handlers/greet.ts";
import { expect, it } from "bun:test";
import { db } from "../../db";
import { greetingHistoryTable } from "../../db/schema/application";
import { type GreetingRequest } from "../../common/schema";

describe("greet", () => {
    beforeEach(async () => {
        await createDB();
    });

    afterEach(async () => {
        await resetDB();
    });
    
    
    it("should return a greeting with just the name", async () => {
      const input: GreetingRequest = { name: "Alice" };
      const greeting = await greet(input);
      expect(greeting).toEqual("Hello, Alice!");
    });

    
    
    it("should store the greeting in history", async () => {
      const input: GreetingRequest = { name: "Bob" };
      await greet(input);
      
      const history = await db.select().from(greetingHistoryTable).execute();
      exp

In [14]:
print(handler_solution.data.output.feedback["stdout"])

None


In [15]:
print(handler_solution.data.output.handler)

import { db } from "../db";
import { greet, type GreetingRequest, type GreetingResponse } from "../common/schema";
import { greetingRequestsTable, greetingResponsesTable } from "../db/schema/application";

export const handle: typeof greet = async (options: GreetingRequest): Promise<GreetingResponse> => {
    // Insert the greeting request
    const [requestRecord] = await db.insert(greetingRequestsTable)
        .values({
            name: options.name
        })
        .returning();

    // Generate greeting message
    const message = `Hello, ${options.name}!`;

    // Store the response
    await db.insert(greetingResponsesTable)
        .values({
            request_id: requestRecord.id,
            message: message
        });

    return {
        message: message
    };
};


In [ ]:
queue = [ts_root]
while queue:
    node = queue.pop(0)
    if isinstance(node.data.output, Exception):
        print(node.data.output)
    elif node.data.output.feedback['exit_code'] != 0:
        print(node.data.output.feedback["stdout"])
    queue.extend(node.children)

In [ ]:
with open("templates/interpolation/handler_test.tpl", "r") as f:
    handler_test_tpl = jinja_env.from_string(f.read())

tests = [
    """
it("should return a greeting", async () => {
    const input: GreetingRequest = { name: "Alice", greeting: "Hello" };
    const greeting = await greet(input);
    expect(greeting).toEqual("Hello, Alice!");
});
""".strip(),
    """
it("should store the greeting request", async () => {
    const input: GreetingRequest = { name: "Alice", greeting: "Hello" };
    await greet(input);
    const requests = await db.select().from(greetingRequestsTable).execute();
    expect(requests).toHaveLength(1);
    expect(requests[0].name).toEqual("Alice");
    expect(requests[0].greeting).toEqual("Hello");
});
""".strip(),
]

imports = """
import { expect, it } from "bun:test";
import { db } from "../../db";
import { greetingRequestsTable } from "../../db/schema/application";
import { type GreetingRequest } from "../../common/schema";
""".strip()

params = {
    "handler_name": "greet",
    "handler_function_import": 'import { greet } from "../../common/schema"',
    "imports": imports,
    "tests": tests,
}

print(handler_test_tpl.render(params))